In [1]:
#TDA API testing

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import math
import datetime

In [4]:
import tda

In [ ]:
import

In [5]:
tda

<module 'tda' from '/Users/luke/opt/anaconda3/lib/python3.7/site-packages/tda/__init__.py'>

In [ ]:
# Testing the client authentication server:

tda